### **Key Libraries:**
* **os and json:** Standard Python libraries for managing environment variables and handling JSON data.
* **dotenv:** Loads environment variables from a .env file, allowing secure API key handling.
* **OpenAI:** Provides the OpenAI API’s client for interacting with language models.
* **Gradio:** A framework for creating web interfaces for ML models, useful for quick and user-friendly AI demos.

In [50]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [66]:
# Initialization

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [52]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [53]:
def chat(message, history):
    messages = [{"role" : "system", "content" : system_message}]
    for human, assistant in history:
        messages.append({"role" : "user", "content" : human})
        messages.append({"role" : "assistant", "content" : assistant})
    messages.append({"role" : "user", "content" : message})
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

In [54]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [55]:
get_ticket_price("London")

Tool get_ticket_price called for London


'$799'

In [56]:
get_ticket_price("China")

Tool get_ticket_price called for China


'Unknown'

## Getting OpenAI to use our Tool
There's some fiddly stuff to allow OpenAI "to call our tool"

This **price_function** dictionary specifies the function’s details, including its name, description, and parameters. This setup is crucial for an AI model to understand when and how to use the **get_ticket_price** function.

In [57]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    # Identify the function by name, allowing the AI model to recognize and invoke it directly.
    "name" : "get_ticket_price ",
    # "description": Provides an explanation of when and why the function should be used, guiding the model to call it specifically when ticket price information is requested.
    "description" : "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    # "parameters": Defines the structure of inputs required by the function
    "parameters" : {
        "type" : "object", # Specify that the parameters will be in object form
        #"properties":  Lists required fields within the function’s parameters.
        "properties" : {
            # A required field describing the destination for which the user seeks a ticket price.
            "destination_city" : {
            "type" : "string",
            "description" : "The city that the customer wants to travel to",
        },
    },
    # Ensure that destination_city is provided whenever this function is called, as it’s essential for retrieving the price.
    "required" : ["destination_city"],
    # Restrict inputs to just the specified parameters, improving reliability.
    "additionalProperties" : False
    }
}

###  **Significance of this Structure in AI Systems**
* **Tool Functionality:** The AI assistant can reference the **tools** list to check available functions and their usage context. It allows the assistant to dynamically choose and call **get_ticket_price** whenever the user asks a price-related question.
* **Parameter Validation:** By defining required parameters, the assistant ensures each call is complete and accurate, minimizing user misunderstandings.
* **Enhanced User Experience:** Structured descriptions help the model know when each tool is appropriate, improving response accuracy and making interactions smoother for the end user.

In [58]:
# This list of tools provides a standardized way for an AI model to access various functions it might use during interactions.
# "type": "function": Specifies that each item in the list is a function type, indicating the role of the item.
# "function": price_function: Associates the actual function dictionary (price_function) with the tool.
tools = [{"type" : "function", "function" : price_function}]

## Getting OpenAI to use our Tool

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

## **Technical and AI Concepts**

This structured approach enables the AI to interactively use tools in response to user requests, adding functionality that goes beyond basic text generation. 

* **Tool Invocation:** By monitoring **finish_reason** for "tool_calls," the AI can selectively call predefined functions, enabling it to perform dynamic tasks based on context.
* **Re-prompting for Contextual Response:** After handling the tool call, the function makes a second OpenAI API call with updated messages. This technique helps ensure the assistant’s response is well-aligned with the context and tool output.
* **Error Handling Potential:** This structure provides flexibility in handling various tool outputs, but adding checks (e.g., for invalid city names) to **handle_tool_call** could further refine error handling.
  
This structured approach enables the AI to interactively use tools in response to user requests, adding functionality that goes beyond basic text generation. 

In [69]:
def chat(message, history):
    messages = [{"role" : "system", "content" : system_message}]
    for human, assistant in history:
        messages.append({"role" : "user", "content" : human})
        messages.append({"role" : "assistant", "content" : assistant})
    messages.append({"role" : "user", "content" : message})

    # Call the OpenAI API with tools as an additional parameter, enabling the assistant to invoke specific functions if needed (like get_ticket_price).
    # The tools parameter allows the assistant to check whether a tool (function) should be called based on user input.
    response = openai.chat.completions.create(model = MODEL, messages = messages, tools = tools)

    # check if the response’s end reason was a tool call, meaning the assistant identified that a function like get_ticket_price could address the user’s query.
    if response.choice[0].finish_reason == "tool_calls": 
        # Extract the message that triggered the tool call.
        message = response.choices[0].message 
        # Invoke handle_tool_call, which processes the tool call by:
            # ---- Extracting parameters (like the destination city).
            # ----- Calling the relevant function (get_ticket_price).
            # ----- Formatting a response based on the function output.
        response, city = handle_tool_call(message)
        # Append the tool call message and the tool’s response to maintain conversation continuity.
        messages.append(message)
        messages.append(response)
        # Call OpenAI again, providing updated messages (with tool output) to complete the user interaction.
        response = openai.chat.completions.create(model = MODEL, messages = messages)

    # Deliver the final response to the user, incorporating any tool output as needed.
    return response.choices[0].message.content
    

## **How This Fits into the AI Workflow**

This function integrates seamlessly into the **chat** flow, enabling the AI assistant to handle tool-based requests dynamically and return accurate, contextually relevant information to the user.

* **Dynamic Tool Execution:** This function dynamically extracts the required information (destination city), calls the relevant tool (**get_ticket_price**), and prepares the data for the AI model.
* **Clear JSON Structure:** By packaging the response in JSON, it ensures consistent formatting, which aids the AI in understanding and incorporating the tool’s output into its responses.
* **Tool Call Management:** Using the **tool_call_id** ensures that each response is linked back to the specific tool invocation, enhancing traceability and reliability.


In [70]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    # Purpose: Selects the first tool call from the AI’s response (in case of multiple tools, which here we assume to be one).
    # Mechanism: Accesses message.tool_calls[0] to retrieve the necessary tool call information, including arguments and tool call ID.
    tool_call = message.tool_calls[0]
    # Purpose: Extracts the city for which the user wants to know the ticket price.
    # json.loads(tool_call.function.arguments): Parses the tool call arguments (which are in JSON format) to access specific input values.
    arguments = json.loads(tool_call.function.arguments)
    # Retrieve the destination_city value, expected to match a city in the ticket_prices dictionary from earlier.
    city = arguments.get("destination_city")
    # Passes city to get_ticket_price, which returns the ticket price if available, or “Unknown” if the city is not found.
    price = get_ticket_price(city)
    # Construct a structured response that can be incorporated into the AI’s conversational flow.
    response = {
        # Indicate this response comes from a tool, helping the AI to process it correctly in context.
        "role" : "tool", 
        #  Convert the destination city and ticket price data into JSON format for clarity and consistency in the response.
        "content" : json.dumps({"destination_city": city, "price": price}),
        # Associate the response with the specific tool call ID, ensuring accurate linkage between request and response.
        "tool_call_id" : message.tool_calls[0].id
    }
    # Return the constructed response and the city name for potential further use in the chat function.
    # The response dictionary is intended for use in the conversation, while city might be used for logging or additional processing.
    return response, city

In [72]:
gr.ChatInterface(fn = chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "D:\Programms\Lib\site-packages\gradio\queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Programms\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Programms\Lib\site-packages\gradio\blocks.py", line 2014, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Programms\Lib\site-packages\gradio\blocks.py", line 1565, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Programms\Lib\site-packages\gradio\utils.py", line 813, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Programms\Lib\site-packages\gradio\chat_interface.py", line 638, in _subm